In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

In [2]:
df_train = pd.read_csv("data_train.csv")
df_train

,Ball_Possession_Home_local,Ball_Possession_Away_visitante,Shots_on_Goal_Home_local,Shots_on_Goal_Away_visitante,Fouls_Home_local,Fouls_Away_visitante,Yellow_Cards_Home_local,Yellow_Cards_Away_visitante,Red_Cards_Home_local,Red_Cards_Away_visitante,...,expected_goals_xg_away_visitante,Goalkeeper_Saves_Home_local,Goalkeeper_Saves_Away_visitante,Offsides_Home_local,Offsides_Away_visitante,Total_Passes_Home_local,Total_Passes_Away_visitante,save_ratio_home_local,save_ratio_away_visitante,resultado
0,51.0,49.0,8.0,0.0,8.0,11.0,0.0,0.0,0.0,0.0,...,0.51,0.0,3.0,1.0,0.0,456.0,432.0,0.000000,0.375000,0
1,41.0,59.0,5.0,4.0,15.0,16.0,4.0,4.0,0.0,0.0,...,1.56,3.0,3.0,0.0,6.0,283.0,427.0,0.750000,0.600000,0
2,34.0,66.0,4.0,3.0,12.0,8.0,4.0,0.0,0.0,0.0,...,0.60,3.0,3.0,3.0,0.0,323.0,635.0,1.000000,0.750000,0
3,65.0,35.0,5.0,4.0,9.0,11.0,0.0,0.0,0.0,0.0,...,1.50,3.0,4.0,2.0,1.0,593.0,329.0,0.750000,0.800000,1
4,66.0,34.0,3.0,4.0,9.0,13.0,0.0,2.0,0.0,0.0,...,0.46,3.0,2.0,2.0,2.0,538.0,276.0,0.750000,0.666667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8895,71.0,29.0,12.0,3.0,11.0,12.0,2.0,2.0,0.0,0.0,...,1.26,2.0,8.0,2.0,3.0,605.0,234.0,0.666667,0.666667,0
8896,41.0,59.0,9.0,2.0,12.0,6.0,0.0,2.0,0.0,0.0,...,1.56,1.0,9.0,6.0,3.0,366.0,556.0,0.500000,1.000000,2
8897,33.0,67.0,1.0,8.0,18.0,11.0,3.0,0.0,0.0,0.0,...,1.58,7.0,1.0,2.0,2.0,262.0,562.0,0.875000,1.000000,2
8898,79.0,21.0,7.0,2.0,12.0,12.0,2.0,2.0,0.0,0.0,...,1.12,1.0,5.0,2.0,1.0,769.0,200.0,0.500000,0.714286,0


In [3]:
df_test_model = pd.read_csv("data_test.csv")
df_test_model

,resultado,Ball_Possession_Home_local,Shots_on_Goal_Home_local,Fouls_Home_local,Yellow_Cards_Home_local,Red_Cards_Home_local,expected_goals_xg_home_local,Goalkeeper_Saves_Home_local,Offsides_Home_local,Total_Passes_Home_local,...,Ball_Possession_Away_visitante,Shots_on_Goal_Away_visitante,Fouls_Away_visitante,Yellow_Cards_Away_visitante,Red_Cards_Away_visitante,expected_goals_xg_away_visitante,Goalkeeper_Saves_Away_visitante,Offsides_Away_visitante,Total_Passes_Away_visitante,save_ratio_away_visitante
0,0,59.552573,6.000000,4.487696,0.664430,0.029083,2.389933,2.760626,0.970917,546.713647,...,39.666667,3.000000,13.111111,2.666667,0.000000,1.013333,4.111111,2.222222,355.666667,0.674868
1,2,45.770833,4.625000,9.604167,1.666667,0.083333,1.503333,2.979167,1.708333,392.125000,...,48.000000,5.100000,10.000000,2.300000,0.000000,1.420000,4.700000,1.800000,437.900000,0.676429
2,2,52.822581,4.419355,11.024194,1.701613,0.048387,1.865242,2.508065,2.016129,482.040323,...,46.800000,4.600000,11.900000,2.400000,0.000000,1.338000,4.300000,1.800000,447.500000,0.644643
3,2,43.592593,3.654321,9.098765,1.419753,0.006173,1.089630,3.092593,2.277778,370.561728,...,44.222222,4.555556,11.777778,2.222222,0.000000,1.282222,2.444444,1.555556,399.222222,0.576455
4,0,60.128668,5.975169,4.600451,0.683973,0.022573,1.923747,2.616253,0.984199,591.882619,...,41.800000,4.200000,14.200000,2.400000,0.100000,1.496000,4.000000,2.100000,350.400000,0.677778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,0,50.221154,4.060897,2.875000,0.480769,0.016026,1.265545,2.788462,0.583333,467.035256,...,56.200000,6.100000,10.400000,1.200000,0.000000,1.908000,1.300000,2.000000,560.600000,0.616667
185,0,57.404063,5.934537,4.832957,0.731377,0.015801,1.683747,2.636569,0.866817,545.176072,...,41.800000,4.200000,14.200000,2.400000,0.100000,1.496000,4.000000,2.100000,350.400000,0.677778
186,0,38.785714,3.678571,11.642857,2.392857,0.071429,1.117500,2.785714,1.928571,342.821429,...,46.800000,4.600000,11.900000,2.400000,0.000000,1.338000,4.300000,1.800000,447.500000,0.644643
187,0,48.162371,4.090206,4.005155,0.677835,0.025773,1.571933,2.757732,0.551546,415.368557,...,41.700000,4.300000,12.600000,2.500000,0.500000,1.000000,5.600000,2.200000,367.500000,0.662112


In [4]:
X_test = df_test_model.drop(columns=['resultado'])
y_test = df_test_model['resultado']
X_train = df_train.drop(columns=['resultado'])
y_train = df_train['resultado']


In [5]:
X_test = df_test_model[X_train.columns]

### LightGBMClassifier Restrictivo

In [9]:
lgbm = LGBMClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05],
    'max_depth': [3, 5, 8],
    'num_leaves': [15, 30],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}

grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)
mejor_lgbm = grid_search.best_estimator_

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0,000365 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1091
[LightGBM] [Info] Number of data points in the train set: 8900, number of used features: 20
[LightGBM] [Info] Start training from score -0,777017
[LightGBM] [Info] Start training from score -1,395777
[LightGBM] [Info] Start training from score -1,229003


In [11]:
# Evaluación en entrenamiento
y_train_pred = mejor_lgbm.predict(X_train)
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Train F1 Score (macro):", f1_score(y_train, y_train_pred, average='macro'))
print("\nTrain Classification Report:")
print(classification_report(y_train, y_train_pred))

Train Accuracy: 0.9896629213483146
Train F1 Score (macro): 0.9887045559814056

Train Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4092
           1       0.99      0.98      0.98      2204
           2       0.99      0.99      0.99      2604

    accuracy                           0.99      8900
   macro avg       0.99      0.99      0.99      8900
weighted avg       0.99      0.99      0.99      8900



In [12]:
# Evaluación en test
y_pred = mejor_lgbm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score (macro):", f1_score(y_test, y_pred, average='macro'))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# ROC AUC Multiclase
y_proba_test = mejor_lgbm.predict_proba(X_test)
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
roc_auc = roc_auc_score(y_test_bin, y_proba_test, multi_class='ovr')
print("ROC AUC Score (One-vs-Rest):", roc_auc)


Accuracy: 0.5343915343915344
F1 Score (macro): 0.5087562074645889

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.65      0.67        86
           1       0.33      0.59      0.42        46
           2       0.69      0.32      0.43        57

    accuracy                           0.53       189
   macro avg       0.57      0.52      0.51       189
weighted avg       0.60      0.53      0.54       189

ROC AUC Score (One-vs-Rest): 0.747789309182829


### Eliminar columnas

In [13]:
# Eliminar features del dataframe original
features_to_remove = [
    'save_ratio_away_visitante',
    'save_ratio_home_local',
    'Shots_on_Goal_Home_local',
    'Goalkeeper_Saves_Away_visitante'
]

df_train = df_train.drop(columns=features_to_remove)
df_test_model = df_test_model.drop(columns=features_to_remove)


In [14]:
X_test = df_test_model.drop(columns=['resultado'])
y_test = df_test_model['resultado']
X_train = df_train.drop(columns=['resultado'])
y_train = df_train['resultado']

In [ ]:
X_test = df_test_model[X_train.columns]

In [16]:
lgbm = LGBMClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05],
    'max_depth': [3, 5, 8],
    'num_leaves': [15, 30],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}

grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)
mejor_lgbm = grid_search.best_estimator_

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0,000341 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 986
[LightGBM] [Info] Number of data points in the train set: 8900, number of used features: 16
[LightGBM] [Info] Start training from score -0,777017
[LightGBM] [Info] Start training from score -1,395777
[LightGBM] [Info] Start training from score -1,229003
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [17]:
# Evaluación en entrenamiento
y_train_pred = mejor_lgbm.predict(X_train)
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Train F1 Score (macro):", f1_score(y_train, y_train_pred, average='macro'))
print("\nTrain Classification Report:")
print(classification_report(y_train, y_train_pred))

Train Accuracy: 0.7815730337078651
Train F1 Score (macro): 0.7424812098140867

Train Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.92      0.84      4092
           1       0.80      0.43      0.56      2204
           2       0.80      0.86      0.83      2604

    accuracy                           0.78      8900
   macro avg       0.79      0.74      0.74      8900
weighted avg       0.79      0.78      0.77      8900



In [18]:
# Evaluación en test
y_pred = mejor_lgbm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score (macro):", f1_score(y_test, y_pred, average='macro'))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# ROC AUC Multiclase
y_proba_test = mejor_lgbm.predict_proba(X_test)
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
roc_auc = roc_auc_score(y_test_bin, y_proba_test, multi_class='ovr')
print("ROC AUC Score (One-vs-Rest):", roc_auc)


Accuracy: 0.5026455026455027
F1 Score (macro): 0.4249882519343597

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.55      0.60        86
           1       0.44      0.09      0.15        46
           2       0.40      0.77      0.53        57

    accuracy                           0.50       189
   macro avg       0.51      0.47      0.42       189
weighted avg       0.53      0.50      0.47       189

ROC AUC Score (One-vs-Rest): 0.6748643870144662


### Escalamos variables

In [20]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
lgbm = LGBMClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05],
    'max_depth': [3, 5, 8],
    'num_leaves': [15, 30],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}

grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train_scaled, y_train)
mejor_lgbm = grid_search.best_estimator_

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0,000813 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 998
[LightGBM] [Info] Number of data points in the train set: 8900, number of used features: 16
[LightGBM] [Info] Start training from score -0,777017
[LightGBM] [Info] Start training from score -1,395777
[LightGBM] [Info] Start training from score -1,229003
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [24]:
# Evaluación en entrenamiento
y_train_pred = mejor_lgbm.predict(X_train_scaled)
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Train F1 Score (macro):", f1_score(y_train, y_train_pred, average='macro'))
print("\nTrain Classification Report:")
print(classification_report(y_train, y_train_pred))

c:\Users\fer_r\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Train Accuracy: 0.7819101123595505
Train F1 Score (macro): 0.7442851850498643

Train Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.92      0.84      4092
           1       0.79      0.44      0.57      2204
           2       0.80      0.86      0.83      2604

    accuracy                           0.78      8900
   macro avg       0.79      0.74      0.74      8900
weighted avg       0.78      0.78      0.77      8900



In [25]:
# Evaluación en test
y_pred = mejor_lgbm.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score (macro):", f1_score(y_test, y_pred, average='macro'))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# ROC AUC Multiclase
y_proba_test = mejor_lgbm.predict_proba(X_test)
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
roc_auc = roc_auc_score(y_test_bin, y_proba_test, multi_class='ovr')
print("ROC AUC Score (One-vs-Rest):", roc_auc)


Accuracy: 0.4497354497354497
F1 Score (macro): 0.37493805984372025

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.48      0.52        86
           1       0.33      0.07      0.11        46
           2       0.38      0.72      0.50        57

    accuracy                           0.45       189
   macro avg       0.43      0.42      0.37       189
weighted avg       0.45      0.45      0.41       189

ROC AUC Score (One-vs-Rest): 0.5634702376629286


c:\Users\fer_r\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
